In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import random

In [2]:
coupon_list_train = pd.read_csv('input/coupon_list_train.csv')
coupon_list_test = pd.read_csv('input/coupon_list_test.csv')
user_list = pd.read_csv('input/user_list.csv')
coupon_purchases_train = pd.read_csv("input/coupon_detail_train.csv")

coupon_area_train = pd.read_csv('input/coupon_area_train.csv')
coupon_area_test = pd.read_csv('input/coupon_area_test.csv')
coupon_visit_train = pd.read_csv('input/coupon_visit_train.csv')

In [3]:
purchased_coupons_train = coupon_purchases_train.merge(coupon_list_train, on='COUPON_ID_hash', how='inner')

In [4]:
features = ['COUPON_ID_hash', 'USER_ID_hash',
            'GENRE_NAME', 'DISCOUNT_PRICE', 'PRICE_RATE',
            'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
            'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
            'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']

In [5]:
purchased_coupons_train = purchased_coupons_train[features]

In [6]:
# 10000 ROWS
df = purchased_coupons_train.copy()
df = df[:10000] 

In [7]:
df['DISCOUNT_PRICE'] = 1 / np.log10(df['DISCOUNT_PRICE'])
df['PRICE_RATE'] = (df['PRICE_RATE'] / 100) ** 2

In [8]:
df_temp = df.copy()

categorical_cols = ['COUPON_ID_hash', 'USER_ID_hash', 'GENRE_NAME', 'large_area_name', 'ken_name', 'small_area_name']

In [9]:
df_temp.head(3)

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
0,34c48f84026e08355dc3bd19b427f09a,d9dca3cb44bab12ba313eaa681f663eb,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
1,34c48f84026e08355dc3bd19b427f09a,dd8a79645e8170db6a91bad1d89b4055,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
2,34c48f84026e08355dc3bd19b427f09a,658249379aaaf2b2320061e5218c0c2c,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺


In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_temp[categorical_cols] = df_temp[categorical_cols].apply(lambda col: le.fit_transform(col))
df_temp = df_temp.fillna(1)
df_temp.head(3)

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
0,12,6643,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
1,12,6761,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
2,12,3072,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9


In [11]:
l = df_temp.shape[0]
f = len(df_temp.columns)
t = AnnoyIndex(f, 'angular')

In [12]:
for i in range(l):
    v = [df_temp.values[i][z] for z in range(f)]
    t.add_item(i, v)

In [13]:
t.build(50)

True

In [14]:
#sim = t.get_nns_by_item(0, 5, include_distances=True)
sim = t.get_nns_by_item(0, 5)


In [15]:
df_temp.loc[sim]

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
25,12,6393,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
33,12,6428,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
60,12,6463,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
22,12,6604,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
1,12,6761,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9


In [16]:
np.mean(df_temp.loc[sim].to_numpy(), axis=0)

array([1.20000000e+01, 6.52980000e+03, 8.00000000e+00, 3.12765796e-01,
       6.08400000e-01, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 4.00000000e+00, 7.00000000e+00,
       9.00000000e+00])